In [2]:
# --- Imports ---
import os
import glob
import re
from datetime import datetime
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
from PIL import Image, ImageDraw, ImageFont
import imageio
import warnings
import csv
import platform # To help find system fonts

# --- Add Geopandas and Rasterio features ---
try:
    import geopandas as gpd
    from rasterio.features import rasterize
    from rasterio.transform import from_bounds
    GEOPANDAS_AVAILABLE = True
except ImportError:
    GEOPANDAS_AVAILABLE = False
    print("WARNING: Geopandas not found. Land masking feature will be disabled.")


# --- Helper function to parse date from filename ---
def get_date_from_filename(filename):
    match = re.search(r'_(\d{4})_(\d{2})\.tif$', os.path.basename(filename))
    if match: year, month = map(int, match.groups()); return datetime(year, month, 1)
    else: print(f"Warning: Could not parse date from filename: {filename}. Skipping."); return None

# --- Main Function ---
def create_nightlight_timelapse_and_graph(
    input_folder: str,
    output_path_base: str,
    output_format: str = 'gif',
    cmap_name: str = 'plasma',
    fps: int = 6,
    normalize_animation: bool = False,
    mask_sea: bool = True,
    land_shapefile_path: str = None,
    sea_mask_color: tuple = (0, 0, 0, 0),
    mp4_crf: int = 23,
    text_size_category: str = 'medium',   # <<< New: 'small', 'medium', 'large'
    font_path: str = None,                # <<< New: Optional path to a TTF font
    graph_title: str = 'Average Night Light Intensity Over Time',
    watermark_text: str = 'My Custom Watermark',
    # watermark_size is now determined automatically
    watermark_position: tuple = (10, 10)
    ):
    """
    Creates timelapse, graph, and CSV from GeoTIFFs, with land masking,
    MP4 quality control, and automatic text sizing.

    Args:
        # ... (previous args) ...
        mp4_crf: Constant Rate Factor for MP4 export (lower=better quality/larger size). Default 23.
        text_size_category: Size category for watermark/date text ('small', 'medium', 'large').
                            Size is calculated relative to image height. Requires a TTF
                            font to be found for accurate sizing. Default 'medium'.
        font_path: Optional path to a specific .ttf font file to use for text.
                   If None, searches common system locations. If no TTF found,
                   falls back to default PIL font with limited size control.
        # ... (other args) ...
        # watermark_size: Removed, now calculated automatically.
    """

    print(f"Starting analysis for folder: {input_folder}")
    print(f"Output base: {output_path_base}")
    print(f"Format: {output_format}, FPS: {fps}, Colormap: {cmap_name}")
    print(f"Normalize Animation Frames: {normalize_animation}")
    print(f"Mask Sea Areas: {mask_sea}")
    if output_format.lower() == 'mp4': print(f"MP4 CRF Value: {mp4_crf}")
    print(f"Text Size Category: {text_size_category}") # <<< Log text size category

    # --- Validate Inputs for Land Masking ---
    if mask_sea and not GEOPANDAS_AVAILABLE: print("WARNING: Geopandas not installed. Disabling sea masking."); mask_sea = False
    if mask_sea and not land_shapefile_path: print("WARNING: 'land_shapefile_path' not provided. Disabling sea masking."); mask_sea = False
    if mask_sea and not os.path.exists(land_shapefile_path): print(f"WARNING: Land shapefile not found at '{land_shapefile_path}'. Disabling sea masking."); mask_sea = False


    # --- 1. Find and Sort TIFF Files ---
    # ... (File finding/sorting logic remains identical) ...
    search_pattern = os.path.join(input_folder, '*.tif')
    tif_files = glob.glob(search_pattern)
    if not tif_files: print(f"Error: No .tif files found in {input_folder}"); return
    file_date_pairs = [(get_date_from_filename(f), f) for f in tif_files]; file_date_pairs = [p for p in file_date_pairs if p[0] is not None]
    if not file_date_pairs: print(f"Error: Could not parse dates from any filenames in {input_folder}"); return
    file_date_pairs.sort(); sorted_files = [f[1] for f in file_date_pairs]; sorted_dates = [f[0] for f in file_date_pairs]; date_labels = [d.strftime('%Y-%m') for d in sorted_dates]
    print(f"Found and sorted {len(sorted_files)} TIFF files.")


    # --- Global Variables & Load Raster Metadata ---
    raster_transform = None; raster_shape = None; raster_crs = None
    land_mask_array = None; land_mask_available = False
    try:
        with rasterio.open(sorted_files[0]) as src:
            raster_transform = src.transform; raster_shape = src.shape; raster_crs = src.crs
            print(f"Raster Properties: Shape={raster_shape}, CRS={raster_crs}")
    except Exception as e: print(f"ERROR: Could not read metadata from first raster file: {e}"); return


    # --- Load Font & Calculate Size ---
    font = None
    calculated_font_size = 10 # Default/fallback size
    ttf_font_found = False
    print("\nAttempting to load font...")

    def find_system_font(font_names):
        system = platform.system()
        if system == "Windows":
            font_dir = os.path.join(os.environ.get("SystemRoot", "C:\\Windows"), "Fonts")
        elif system == "Linux": # Common Linux paths
             font_dirs = ["/usr/share/fonts/truetype/dejavu", "/usr/share/fonts/truetype/msttcorefonts", "/usr/share/fonts/truetype/liberation", "/usr/share/fonts/truetype", "/usr/local/share/fonts", os.path.expanduser("~/.fonts")]
        elif system == "Darwin": # macOS
             font_dirs = ["/System/Library/Fonts", "/Library/Fonts", os.path.expanduser("~/Library/Fonts")]
        else:
            font_dirs = []

        for name in font_names:
            # Try direct name first (Windows often registers them)
            try:
                 # Check if Pillow can find it by name directly (esp. on Windows)
                 temp_font = ImageFont.truetype(name, 10)
                 print(f"  Found font '{name}' by name.")
                 return name # Return name if found by Pillow
            except IOError:
                 pass # Continue searching paths

            # Search in common directories
            for directory in font_dirs:
                 # Try common extensions
                 for ext in ['.ttf', '.otf']:
                      potential_path = os.path.join(directory, name + ext)
                      if os.path.exists(potential_path):
                           print(f"  Found font at: {potential_path}")
                           return potential_path
                 # Sometimes font files have different casing or full names
                 try:
                     for fname in os.listdir(directory):
                         if name.lower() in fname.lower() and (fname.lower().endswith(".ttf") or fname.lower().endswith(".otf")):
                             potential_path = os.path.join(directory, fname)
                             print(f"  Found potential font match: {potential_path}")
                             return potential_path
                 except OSError:
                     continue # Directory might not be accessible
        return None # Font not found

    # 1. Try user-provided path
    if font_path and os.path.exists(font_path):
        try:
            # Calculate size based on image height and category
            # Base size aims for roughly 1/50th of image height, with minimum of 12
            base_size = max(12, int(raster_shape[0] / 50))
            size_multipliers = {'small': 0.7, 'medium': 1.0, 'large': 1.4}
            multiplier = size_multipliers.get(text_size_category.lower(), 1.0)
            calculated_font_size = int(base_size * multiplier)
            font = ImageFont.truetype(font_path, calculated_font_size)
            print(f"  Loaded specified TTF font: {font_path} with size {calculated_font_size}")
            ttf_font_found = True
        except IOError as e:
            print(f"  Warning: Could not load specified font '{font_path}': {e}")
        except Exception as e:
             print(f"  Warning: Unexpected error loading specified font '{font_path}': {e}")


    # 2. If no user path or failed, try common system fonts
    if not ttf_font_found:
        common_fonts_to_try = ['DejaVuSans', 'arial', 'Arial', 'LiberationSans-Regular'] # Add more if needed
        found_font_path = find_system_font(common_fonts_to_try)

        if found_font_path:
            try:
                base_size = max(12, int(raster_shape[0] / 50))
                size_multipliers = {'small': 0.7, 'medium': 1.0, 'large': 1.4}
                multiplier = size_multipliers.get(text_size_category.lower(), 1.0)
                calculated_font_size = int(base_size * multiplier)
                font = ImageFont.truetype(found_font_path, calculated_font_size)
                print(f"  Loaded system TTF font: {found_font_path} with size {calculated_font_size}")
                ttf_font_found = True
            except IOError as e:
                print(f"  Warning: Could not load found system font '{found_font_path}': {e}")
            except Exception as e:
                print(f"  Warning: Unexpected error loading system font '{found_font_path}': {e}")


    # 3. If still no TTF font, fall back to default PIL font
    if not ttf_font_found:
        try:
            font = ImageFont.load_default()
            print("  Loaded default PIL font.")
            print("  WARNING: TTF font not found. Using default PIL font.")
            print("           'small'/'medium'/'large' size setting will have limited effect.")
            # Use a fixed 'medium' size estimate for default font when calculating position
            calculated_font_size = 10 # Default estimate
        except IOError:
            print("ERROR: Could not load default PIL font. Text will not be added.")
            font = None # Ensure font is None if even default fails


    # --- Prepare Land Mask ---
    # ... (Land mask preparation logic remains identical) ...
    if mask_sea:
        print(f"\nPreparing land mask from: {land_shapefile_path}")
        # ... (try-except block for loading, reprojecting, rasterizing GDF) ...
        try:
            land_gdf = gpd.read_file(land_shapefile_path)
            if land_gdf.crs != raster_crs: land_gdf = land_gdf.to_crs(raster_crs)
            land_mask_array = rasterize(shapes=[(geom, 1) for geom in land_gdf.geometry], out_shape=raster_shape, transform=raster_transform, fill=0, dtype='uint8')
            land_mask_array = land_mask_array.astype(bool); land_mask_available = True
            print(f"  Land mask created successfully.")
        except MemoryError: print("ERROR: MemoryError creating land mask."); mask_sea = False
        except Exception as e: print(f"ERROR creating land mask: {e}"); mask_sea = False


    # --- 2. Process Files: Calculate Raw Stats & Determine Range ---
    # ... (Pass 1 logic remains identical) ...
    monthly_avg_intensity = []; all_valid_data_list = []; files_with_valid_data_count = 0; max_raw_value_overall = -np.inf
    print("\n--- Pass 1: Calculating Raw Statistics & Preparing Range ---")
    for i, filepath in enumerate(sorted_files):
        # ... (Inner loop identical: read, mask, calculate stats, store) ...
        try:
            with rasterio.open(filepath) as src:
                raw_data = src.read(1).astype(np.float32)
                with warnings.catch_warnings(): warnings.simplefilter("ignore"); current_max_raw = np.nanmax(raw_data)
                if np.isfinite(current_max_raw) and current_max_raw > max_raw_value_overall: max_raw_value_overall = current_max_raw
                invalid_mask = (np.isnan(raw_data)) | (raw_data <= 0); valid_data = raw_data[~invalid_mask]
                if valid_data.size > 0:
                    files_with_valid_data_count += 1
                    with warnings.catch_warnings(): warnings.simplefilter("ignore"); min_val, max_val, mean_val = np.nanmin(valid_data), np.nanmax(valid_data), np.nanmean(valid_data)
                    if not (np.isfinite(min_val) and np.isfinite(max_val) and np.isfinite(mean_val)): monthly_avg_intensity.append(np.nan)
                    else: monthly_avg_intensity.append(mean_val); all_valid_data_list.append(valid_data)
                else: monthly_avg_intensity.append(np.nan)
        except Exception as e: print(f"  ERROR reading {os.path.basename(filepath)}: {e}"); monthly_avg_intensity.append(np.nan)
    # ... (Pass 1 Summary and Range Determination logic remains identical) ...
    print(f"\n--- Pass 1 Summary ---")
    print(f"  Processed {len(sorted_files)}. Found valid data in {files_with_valid_data_count} files.")
    if not np.isfinite(max_raw_value_overall): max_raw_value_overall = 1.0
    print(f"  Overall maximum raw value encountered: {max_raw_value_overall:.4f}")
    vis_vmin, vis_vmax = 0.0, max(1.0, max_raw_value_overall)
    if all_valid_data_list:
        print("  Determining visualization range using percentiles...")
        try:
            all_valid_data_list_filt = [arr for arr in all_valid_data_list if isinstance(arr, np.ndarray) and arr.size > 0]
            if all_valid_data_list_filt:
                 concatenated_data = np.concatenate(all_valid_data_list_filt); del all_valid_data_list
                 if concatenated_data.size > 0:
                     with warnings.catch_warnings(): warnings.simplefilter("ignore"); p_low = np.nanpercentile(concatenated_data, 0.5); p_high = np.nanpercentile(concatenated_data, 99.5)
                     if np.isfinite(p_low) and np.isfinite(p_high) and p_high > p_low: vis_vmin = p_low; vis_vmax = p_high; print(f"  Using 0.5-99.5 percentile range: {vis_vmin:.4f} - {vis_vmax:.4f}")
                     else: print("  Warning: Percentile calc failed. Using fallback.")
                 del concatenated_data
        except Exception as e: print(f"  ERROR during range calc: {e}")
    if vis_vmax <= vis_vmin: vis_vmax = vis_vmin + 1.0
    if vis_vmin < 0: vis_vmin = 0.0
    if normalize_animation:
        print(f"--- Using NORMALIZED animation range: vmin={vis_vmin:.4f}, vmax={vis_vmax:.4f} ---")
        norm = mcolors.Normalize(vmin=vis_vmin, vmax=vis_vmax)
    else:
        fixed_vmin, fixed_vmax = 0.0, vis_vmax; print(f"--- Using FIXED animation scaling: vmin={fixed_vmin:.4f}, vmax={fixed_vmax:.4f} ---")
        norm = mcolors.Normalize(vmin=fixed_vmin, vmax=fixed_vmax)


    # --- 3. Save Raw Intensity Data to CSV ---
    # ... (CSV saving logic remains identical) ...
    csv_filename = f"{output_path_base}_intensity_data.csv"
    print(f"\nSaving raw average intensity data to {csv_filename}...")
    try:
        with open(csv_filename, 'w', newline='') as csvfile:
            csvwriter = csv.writer(csvfile); csvwriter.writerow(['Date', 'Average_Radiance']); count_written = 0
            for date_obj, intensity in zip(sorted_dates, monthly_avg_intensity):
                if np.isfinite(intensity): csvwriter.writerow([date_obj.strftime('%Y-%m-%d'), f"{intensity:.6f}"]); count_written += 1
            print(f"  Successfully wrote {count_written} data points to CSV.")
    except Exception as e: print(f"  ERROR writing CSV file: {e}")


    # --- 4. Prepare Frames (Pass 2 with Land Masking & Auto Text Size) ---
    frames = []
    cmap = cm.get_cmap(cmap_name)
    # Font object 'font' is already loaded and sized appropriately (or set to default/None)
    print("\n--- Pass 2: Creating animation frames ---")
    if len(sea_mask_color) == 4: sea_mask_color_np = np.array(sea_mask_color[:4]) / 255.0
    else: print("Warning: Invalid sea_mask_color. Using transparent black."); sea_mask_color_np = np.array([0.0, 0.0, 0.0, 0.0])

    for i, filepath in enumerate(sorted_files):
        # print(f"  Processing frame {i+1}/{len(sorted_files)}: {os.path.basename(filepath)}") # Verbose
        try:
            with rasterio.open(filepath) as src:
                data = src.read(1).astype(np.float32)
                nan_mask = (np.isnan(data)) | (data <= 0); data[nan_mask] = np.nan
                data_processed = norm(np.nan_to_num(data, nan=norm.vmin))
                rgba_image = cmap(data_processed)
                if mask_sea and land_mask_available:
                    sea_pixels_mask = ~land_mask_array
                    if np.sum(sea_pixels_mask) > 0: rgba_image[sea_pixels_mask, :] = sea_mask_color_np
                pil_image = Image.fromarray((rgba_image * 255).astype(np.uint8), 'RGBA')
                draw = ImageDraw.Draw(pil_image)

                # Draw Watermark/Date using the pre-loaded 'font' object
                if font:
                    padding = max(5, int(calculated_font_size * 0.5)) # Padding based on font size
                    # Watermark
                    if watermark_text:
                        try:
                             # Use textbbox for positioning if available
                             wm_bbox = draw.textbbox(watermark_position, watermark_text, font=font)
                             # draw.text(watermark_position, watermark_text, fill=(255, 255, 255, 180), font=font) # Simple position
                             # Adjust position slightly based on actual bbox (optional refinement)
                             draw.text((padding, padding), watermark_text, fill=(255, 255, 255, 180), font=font) # Use padding
                        except (AttributeError, TypeError): # Fallback if textbbox fails
                             draw.text((padding, padding), watermark_text, fill=(255, 255, 255, 180), font=font)
                        except Exception as text_err: print(f"Warn: Watermark draw error: {text_err}") # Catch other errors

                    # Date Label (bottom right)
                    date_str = date_labels[i]
                    try:
                        # Estimate size using textbbox if possible
                        date_bbox = draw.textbbox((0,0), date_str, font=font)
                        text_width = date_bbox[2] - date_bbox[0]
                        text_height = date_bbox[3] - date_bbox[1]
                        date_pos = (pil_image.width - text_width - padding, pil_image.height - text_height - padding) # Use padding
                        draw.text(date_pos, date_str, fill=(255, 255, 255, 200), font=font)
                    except (AttributeError, TypeError): # Fallback estimation
                         # Estimate size based on rough char width and calculated_font_size
                         est_char_width = calculated_font_size * 0.6
                         text_width = int(len(date_str) * est_char_width)
                         text_height = int(calculated_font_size * 1.2) # Approx line height
                         date_pos = (pil_image.width - text_width - padding, pil_image.height - text_height - padding) # Use padding
                         try: draw.text(date_pos, date_str, fill=(255, 255, 255, 200), font=font)
                         except Exception as text_err: print(f"Warn: Date draw error: {text_err}")

                frames.append(np.array(pil_image))
        except Exception as e: print(f"  ERROR creating frame for {os.path.basename(filepath)}: {e}")


    # --- 5. Create Animation ---
    # (Logic now includes mp4_crf)
    if frames:
        output_filename_anim = f"{output_path_base}.{output_format.lower()}"
        print(f"\nSaving animation to {output_filename_anim}...")
        print("  INFO: If sea transparency fails in MP4, try output_format='gif'")
        try:
            if output_format.lower() == 'gif':
                 imageio.mimsave(output_filename_anim, frames, format=output_format, duration=int(1000/fps))
            elif output_format.lower() == 'mp4':
                 imageio.mimsave(output_filename_anim, frames, format='FFMPEG',
                                 fps=fps, macro_block_size=16, # Helps some codecs
                                 output_params=[
                                     '-vcodec', 'libx264',
                                     '-crf', str(mp4_crf),     # Apply quality setting
                                     '-preset', 'medium',      # Encoding speed/compression
                                     '-pix_fmt', 'yuva420p'   # Pixel format for alpha
                                 ])
            else:
                 imageio.mimsave(output_filename_anim, frames, format=output_format, fps=fps)
            print("Animation saved successfully.")
        except Exception as e:
            print(f"Error saving animation: {e}")
            print("  Common issues: Check ffmpeg install/codec support (esp. alpha). Adjust mp4_crf if needed.")
    else:
        print("\nNo frames generated, skipping animation saving.")


    # --- 6. Create and Save Graph (using RAW data) ---
    # (Logic remains identical)
    output_filename_graph = f"{output_path_base}_graph.png"
    print(f"\nSaving graph of raw intensity data to {output_filename_graph}...")
    valid_indices = np.isfinite(monthly_avg_intensity); plot_dates = np.array(sorted_dates)[valid_indices]; plot_intensity_raw = np.array(monthly_avg_intensity)[valid_indices]
    print(f"  Number of valid data points for graph: {len(plot_dates)}")
    if len(plot_dates) > 0:
        fig, ax = plt.subplots(figsize=(15, 7))
        ax.plot(plot_dates, plot_intensity_raw, marker='.', linestyle='-', markersize=5, label='Monthly Avg Radiance (Raw)')
        ax.xaxis.set_major_locator(mdates.YearLocator(2)); ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y')); ax.xaxis.set_minor_locator(mdates.MonthLocator(interval=6)); fig.autofmt_xdate(rotation=45)
        ax.set_xlabel("Time"); ax.set_ylabel("Average Radiance (Raw, e.g., nW/cm²/sr)"); ax.set_title(graph_title); ax.legend(); ax.grid(True, which='both', linestyle='--', alpha=0.6); ax.set_ylim(bottom=0); plt.tight_layout()
        try: plt.savefig(output_filename_graph, dpi=300); print("Graph saved successfully.")
        except Exception as e: print(f"Error saving graph: {e}")
        plt.close(fig)
    else: print("No valid intensity data to plot, skipping graph saving.")

    print("\nProcessing finished.")



In [3]:

# --- Example Usage ---
if __name__ == "__main__":
    # --- Configuration ---
    # <<< IMPORTANT: SET THESE PATHS CORRECTLY >>>
    GEE_DOWNLOAD_FOLDER = r'C:/Users/rodri/My Drive/GEE_Nightlights_France'
    OUTPUT_BASE_NAME = r'C:/Users/rodri/Desktop/Nightlights/France_nightlights' 

    # <<< IMPORTANT: SET PATH TO YOUR LAND SHAPEFILE >>>
    # Download from https://www.naturalearthdata.com/downloads/ (e.g., 50m Land)
    # Unzip and provide the path to the .shp file
                     
    LAND_SHP_PATH = "C:/Users/rodri/Desktop/NIghtlights/Natural_Earth_Coastline/ne_10m_land/ne_10m_land.shp"

    # --- Path checks ---
    if not os.path.isdir(GEE_DOWNLOAD_FOLDER): print(f"ERROR: Input folder not found: {GEE_DOWNLOAD_FOLDER}"); exit()
    output_dir = os.path.dirname(OUTPUT_BASE_NAME)
    if not os.path.isdir(output_dir):
         try: os.makedirs(output_dir, exist_ok=True); print(f"Created output directory: {output_dir}")
         except OSError as e: print(f"ERROR: Could not create output directory: {e}"); exit()
    if not os.path.exists(LAND_SHP_PATH) and os.environ.get("SKIP_SHP_CHECK") != "1":
        print(f"ERROR: 10m Land shapefile not found at '{LAND_SHP_PATH}'")
        print("Please update LAND_SHP_PATH or ensure 'mask_sea=False'.")
        # exit()

    # --- Run the function ---
    create_nightlight_timelapse_and_graph(
        input_folder=GEE_DOWNLOAD_FOLDER,
        output_path_base=OUTPUT_BASE_NAME,
        output_format='mp4',        # 'mp4' or 'gif'
        cmap_name='plasma',         # Colormap
        fps=10,                     # Faster animation speed
        normalize_animation=False,  # Color scale choice
        mask_sea=True,              # Use land mask
        land_shapefile_path=LAND_SHP_PATH, # Path to 10m shapefile
        sea_mask_color=(0, 0, 0, 0),  # Transparent sea
        mp4_crf=20,                 # MP4 Quality (higher = smaller file) adjust 18-28+
        text_size_category='medium',# Choose 'small', 'medium', or 'large' for text
        # font_path=r'C:/Windows/Fonts/arial.ttf', # Optional: Specify a font file directly
        graph_title='France: Average Monthly Night Light Intensity (VIIRS DNB)',
        watermark_text='ANAPLIAN.com',
        watermark_position=(45, 45)
    )

Starting analysis for folder: C:/Users/rodri/My Drive/GEE_Nightlights_France
Output base: C:/Users/rodri/Desktop/Nightlights/France_nightlights
Format: mp4, FPS: 15, Colormap: plasma
Normalize Animation Frames: False
Mask Sea Areas: True
MP4 CRF Value: 20
Text Size Category: medium
Found and sorted 155 TIFF files.
Raster Properties: Shape=(2173, 3274), CRS=EPSG:4326

Attempting to load font...
  Found font 'DejaVuSans' by name.
  Loaded system TTF font: DejaVuSans with size 43

Preparing land mask from: C:/Users/rodri/Desktop/NIghtlights/Natural_Earth_Coastline/ne_10m_land/ne_10m_land.shp
  Land mask created successfully.

--- Pass 1: Calculating Raw Statistics & Preparing Range ---

--- Pass 1 Summary ---
  Processed 155. Found valid data in 116 files.
  Overall maximum raw value encountered: 10419.0771
  Determining visualization range using percentiles...
  Using 0.5-99.5 percentile range: 0.0254 - 35.3330
--- Using FIXED animation scaling: vmin=0.0000, vmax=35.3330 ---

Saving raw 

C:\Users\rodri\AppData\Local\Temp\ipykernel_6304\1655752030.py:281: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap(cmap_name)



Saving animation to C:/Users/rodri/Desktop/Nightlights/France_nightlights.mp4...
  INFO: If sea transparency fails in MP4, try output_format='gif'


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (3274, 2173) to (3280, 2176) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Animation saved successfully.

Saving graph of raw intensity data to C:/Users/rodri/Desktop/Nightlights/France_nightlights_graph.png...
  Number of valid data points for graph: 116
Graph saved successfully.

Processing finished.
